In [ ]:
from __future__ import print_function
import os.path
import pandas as pd
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Ao modificar esses escopos, exclua o arquivo token.json.
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

'''Mostra o uso básico da Sheets API.
Imprime valores de uma planilha de amostra.'''
creds = None

'''O arquivo token.json armazena os tokens de acesso e atualização do usuário e é criado 
automaticamente quando o fluxo de autorização é concluído pela primeira vez.'''
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

# Se não houver credenciais (válidas) disponíveis, deixe o usuário efetuar login.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)

# Salva as credenciais para a próxima execução
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    service = build('sheets', 'v4', credentials=creds)

# Chame a API Sheets
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId='1DTt9QS5MWHwXRHnCg3noxZD0-xpLo0zdQFGpNj_laNU',
                            range='Maio!A1:AK').execute()
    values = result.get('values', [])

    df = pd.DataFrame(values)
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    df = df.drop(df.index[0])
    df = df.dropna(subset=['Empresa'])

except HttpError as err:
    print(err)

display(df)